<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CANCER_AI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
from openai import OpenAI
from google.colab import userdata

# ==============================================================================
# 1. API KEY CONFIGURATION & CLIENT SETUP
# ==============================================================================

# Note: Assumes OPENROUTER_API_KEY and DEEPSEEK_API_KEY are stored in Colab Secrets
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
    if not OPENROUTER_API_KEY or not DEEPSEEK_API_KEY:
        raise ValueError("API keys not set in Colab secrets.")

    # 1. OpenRouter Client (for Qwen/Qwen-VL models)
    qwen_client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )

    # 2. DeepSeek Client (for DeepSeek V3.1 Reasoner/Thinking Mode)
    deepseek_client = OpenAI(
        api_key=DEEPSEEK_API_KEY,
        base_url="https://api.deepseek.com",
    )
except Exception as e:
    print(f"Configuration Error: {e}")
    # Initialize mock clients to allow code structure to be examined without keys
    # In a real scenario, this block would exit/fail.
    class MockClient:
        def chat(self):
            return self
        def completions(self):
            return self
        def create(self, **kwargs):
            return type('MockResponse', (object,), {'choices': [type('MockChoice', (object,), {'message': type('MockMessage', (object,), {'content': 'MOCK RESPONSE: API KEY MISSING'}), 'reasoning_content': 'MOCK REASONING'})]})()
    qwen_client = MockClient()
    deepseek_client = MockClient()


# ==============================================================================
# 2. TOOL DEFINITION (RAG/LIBRARIAN)
# ==============================================================================

def query_clinical_knowledge_base(query: str) -> str:
    """
    MOCK: Simulates DeepSeek V3.1 (Non-Thinking Mode) executing a function call
    to a Retrieval-Augmented Generation (RAG) database for up-to-date guidelines.
    """
    print(f"-> RAG Tool: Searching for '{query}'...")

    # Mock data return based on query keyword (simulating real-time lookup)
    if "EGFR" in query:
        return (
            "RAG Evidence: NCCN Guidelines (v3.2025) recommend Osicentib (osimertinib) for "
            "first-line treatment of EGFR-mutant NSCLC. Key mutations: Exon 19 deletion, L858R. "
            "Resistance often involves T790M mutation. (Confidence: High)"
        )
    elif "biopsy" in query or "histology" in query:
        return (
            "RAG Evidence: Standard Pathology Protocol requires IHC staining for CK7, TTF-1, and P40 "
            "to differentiate between adenocarcinoma and squamous cell carcinoma in lung cancer. (Confidence: Medium)"
        )
    else:
        return "RAG Evidence: No specific, current guidelines found. Default to general oncology principles. (Confidence: Low)"


# ==============================================================================
# 3. AGENT CLASSES IMPLEMENTATION
# ==============================================================================

class PathologyRadiologyAgent:
    """Uses Qwen-VL (Multimodal) for image analysis."""
    def __init__(self, client):
        self.client = client
        self.model = "qwen/qwen-vl-chat"

    def analyze_image(self, image_url: str, context: str) -> str:
        prompt_text = (
            f"You are a specialist pathologist. Analyze this image in the context of: {context}. "
            "Describe the key visual findings (e.g., tumor morphology, margin status, cell type). "
            "Return only the structured text findings, suitable for a diagnostic abstract."
        )
        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": image_url}}
                    ]}
                ]
            )
            findings = completion.choices[0].message.content
            print(f"✅ Pathology Agent Findings (Qwen-VL): {findings[:80]}...")
            return findings
        except Exception as e:
            return f"Error in Qwen-VL analysis: {e}"


class ClinicalReasoningAgent:
    """Uses DeepSeek V3.1 (Thinking Mode/Reasoner) for auditable diagnosis."""
    def __init__(self, client):
        self.client = client
        self.model = "deepseek-reasoner"

    def perform_diagnosis(self, patient_data: str, specialist_findings: str, rag_evidence: str) -> dict:
        system_prompt = (
            "You are a senior oncologist. Your task is to perform a comprehensive, step-by-step "
            "differential diagnosis and propose an initial management plan based on all provided data. "
            "You MUST use your internal 'Thinking Mode' (Chain-of-Thought) to show your logical progression "
            "before outputting the final answer. Prioritize the highest probability diagnosis."
        )

        full_prompt = (
            f"--- PATIENT DATA ---\n{patient_data}\n\n"
            f"--- SPECIALIST FINDINGS ---\n{specialist_findings}\n\n"
            f"--- RAG EVIDENCE (Guidelines) ---\n{rag_evidence}\n\n"
            "Generate the Chain-of-Thought reasoning leading to the Final Diagnosis and Plan."
        )

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_prompt}
                ]
            )

            # DeepSeek Reasoner output structure
            reasoning = getattr(response.choices[0].message, "reasoning_content", "Reasoning not explicitly captured.")
            final_answer = response.choices[0].message.content

            print("✅ DeepSeek Reasoning Complete.")
            return {"diagnosis": final_answer, "reasoning_CoT": reasoning}

        except Exception as e:
            return {"diagnosis": f"Error in DeepSeek Reasoner: {e}", "reasoning_CoT": str(e)}


class OrchestratorAgent:
    """Uses Qwen (General/Chat) to coordinate and synthesize the final report."""
    def __init__(self, qwen_client, deepseek_client):
        self.qwen_client = qwen_client
        self.deepseek_agent = ClinicalReasoningAgent(deepseek_client)
        self.pathology_agent = PathologyRadiologyAgent(qwen_client)
        self.model = "qwen/qwen3-max"

    def run_diagnostic_workflow(self, patient_case: dict) -> dict:
        print("--- 🩺 Starting Agentic Cancer Diagnosis Workflow ---")

        # 1. Multimodal Analysis (Qwen-VL)
        pathology_findings = self.pathology_agent.analyze_image(
            image_url=patient_case['image_url'],
            context=patient_case['history']
        )

        # 2. Knowledge Retrieval (DeepSeek V3.1 Non-Thinking Role - via Mock RAG Tool)
        rag_evidence = query_clinical_knowledge_base(patient_case['genomic_details'])
        print(f"✅ RAG Evidence Retrieved: {rag_evidence[:80]}...")

        # 3. Core Reasoning (DeepSeek V3.1 Thinking Mode)
        reasoning_output = self.deepseek_agent.perform_diagnosis(
            patient_data=patient_case['history'] + " " + patient_case['genomic_details'],
            specialist_findings=pathology_findings,
            rag_evidence=rag_evidence
        )

        # 4. Final Synthesis (Qwen)
        final_report_prompt = (
            f"Synthesize the following information into a professional, clear, and explainable medical report. "
            f"Diagnosis: {reasoning_output['diagnosis']}. Specialist Findings: {pathology_findings}. "
            f"Evidence: {rag_evidence}\n"
            "Ensure the final report is suitable for a clinician, is brief, and includes a confidence score (1-5)."
        )

        try:
            completion = self.qwen_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a professional medical report generator."},
                    {"role": "user", "content": final_report_prompt}
                ]
            )
            final_report = completion.choices[0].message.content

        except Exception as e:
            final_report = f"Error during final report synthesis: {e}"

        return {
            "final_report": final_report,
            "auditable_reasoning_cot": reasoning_output['reasoning_CoT']
        }


# ==============================================================================
# 4. EXECUTION
# ==============================================================================

# --- Mock Patient Case ---
patient_case = {
    "history": "55-year-old male with new onset cough, hemoptysis, and 20lb weight loss over 3 months. Chest X-ray showed a right upper lobe mass.",
    "genomic_details": "Initial blood test indicates positive EGFR mutation (L858R).",
    # Example Lung Cancer Image URL (replace with actual clinical image if available/permitted)
    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/Non-small_cell_lung_cancer.jpg/800px-Non-small_cell_lung_cancer.jpg"
}

# --- Initialize and Run ---
orchestrator = OrchestratorAgent(qwen_client, deepseek_client)
diagnostic_result = orchestrator.run_diagnostic_workflow(patient_case)

print("\n" + "="*80)
print("              FINAL AGENTIC CANCER DIAGNOSTIC REPORT (Synthesized by Qwen)")
print("="*80)
print(diagnostic_result["final_report"])

print("\n" + "-"*80)
print("   AUDIT TRAIL: DeepSeek V3.1 Clinical Reasoning (Chain-of-Thought)")
print("-"*80)
print(diagnostic_result["auditable_reasoning_cot"])

--- 🩺 Starting Agentic Cancer Diagnosis Workflow ---
-> RAG Tool: Searching for 'Initial blood test indicates positive EGFR mutation (L858R).'...
✅ RAG Evidence Retrieved: RAG Evidence: NCCN Guidelines (v3.2025) recommend Osicentib (osimertinib) for fi...
✅ DeepSeek Reasoning Complete.

              FINAL AGENTIC CANCER DIAGNOSTIC REPORT (Synthesized by Qwen)
**Medical Report**

**Patient:** 55-year-old male  
**Presenting Symptoms:** New-onset cough, hemoptysis, and unintentional weight loss (~20 lbs over 3 months).  

**Diagnostic Findings:**  
- Chest X-ray: Right upper lobe mass.  
- Liquid biopsy: Positive for EGFR L858R mutation.  

**Final Diagnosis:**  
Advanced non-small cell lung cancer (NSCLC), adenocarcinoma subtype, EGFR L858R-positive, likely Stage IV.  

**Rationale:**  
The constellation of red-flag symptoms (hemoptysis, weight loss, persistent cough), imaging findings (right upper lobe mass), and detection of an oncogenic EGFR L858R mutation strongly supports a diagno